In [2]:
import numpy as np 
import scipy
import scipy.signal
import scipy.io.wavfile
import matplotlib.pyplot as plt

In [3]:
def gaussian(data, mean, cov, reshape=False):
    if reshape:
        data=np.reshape(data, (len(data),1))
    zero_mean = data - mean
    cov_inv = np.linalg.inv(cov)
    temp =  -0.5 *(zero_mean.T.dot(cov_inv).dot(zero_mean))
    determinant = np.linalg.det(cov)
    numerator = np.exp(temp)
    denomenator = np.sqrt(2*np.pi*determinant)
    gaussian_prob = numerator/denomenator 
    return gaussian_prob

In [4]:
def PCA(x, k):
   # Obtain column wise zero-mean
    '''If the input mean is not zero, bad things happen, we know this from lecture! 
    Therefore, we need to make sure that our data is zero-mean, 
    so instead of using 𝐗 we use 𝐗_mean'''
    means = np.mean(x, axis=1)
    means = np.reshape(means, (means.shape[0], 1))
    x_mean = x - means
   
    # Get the covariance of our zero-mean matrix
    cov_mat = np.cov(x_mean)
    '''As we were told by the TA,  Doing an eigen decomposition on the covariance matrix 
    (instead of doing it directly on the data matrix 𝐗, by SVD) is a easier starting point''' 

    eigenVal,eigenVec = scipy.sparse.linalg.eigsh(cov_mat, k = k)
#     print("Eigen Vectors shape : " + str(eigenVec.shape))
    
    #Calculate feature matrix
    w = np.linalg.inv(np.diag(np.sqrt(eigenVal))).dot(eigenVec.T)
    
    # Calculate weight matrix
    z = np.dot(w, x_mean)
    
    return w, z,means

In [5]:
def find_mean_and_cov(x):
# returns mean and covariance for given sample class
    mean = np.mean(x, axis=1)
    mean = np.reshape(mean, (len(mean),1))
    zero_mean = x - mean
    cov = zero_mean.dot(zero_mean.T) / (zero_mean.shape[1]-1)
    return mean, cov

In [6]:
def chunkify(sample):
    #10260 is the number of rows thats freq x 20. 
    #I'm just making a zeros matrix to be able to hstack/vstack conveniniently, and discard the first column before returning.

    newMat = np.zeros((51300,1))
#     rem = 1300 - sample.shape[1]
#     zeros = np.zeros((rem, 513))
    chunked = []
#     sample = sample.T
#     sample = np.vstack((sample, zeros))
#     sample = sample.T
    for colNum in range(0, np.shape(sample)[1], 100):
        matrix = sample[:,colNum:colNum+100]
        matrix = matrix.flatten('F')
        matrix = np.reshape(matrix, (matrix.shape[0], 1))
        #print("Chunkify matrix shape", matrix.shape)
        newMat = np.hstack((newMat, matrix))
    return newMat[:,1:]

In [40]:
path = "/Users/sushazad/Documents/GitHub/ISSE_cleaned/"
filename = "birdAutoGen22050.wav"
sample, bsound = scipy.io.wavfile.read(path+filename)
print(sample)
print(bsound.shape)
filename = "ZuzanaBirdzClean.wav"
sample2, bsound2 = scipy.io.wavfile.read(path+filename)
print(sample2)
print(bsound2.shape)
bsound2 = np.reshape(bsound2, (bsound2.shape[0],1))
bsound = np.reshape(bsound, (bsound.shape[0],1))
print(bsound.shape)
print(bsound2.shape)
print("Third!!!!")
filename = "GGA_DBirdsOnly.wav"
sample3, bsound3 = scipy.io.wavfile.read(path+filename)
print(sample3)
print(bsound3.shape)
bsound3 = np.reshape(bsound3, (bsound3.shape[0],1))
totBirds = np.vstack((bsound, bsound2, bsound3))[:,0]
print("Total birds:",totBirds.shape)
bf, bt, bZxx = scipy.signal.stft(totBirds, fs=sample,window='hann', noverlap=768, nperseg=1024)
bmag = np.abs(bZxx)
blog = np.log(bmag)
print(blog.shape)
blog=blog[:,:50900]
birdsAloneLog = blog

22050
(3834495,)
22050
(1306264,)
(3834495, 1)
(1306264, 1)
Third!!!!
44100
(7893900,)
Total birds: (13034659,)
(513, 50918)


In [41]:
path = "/Users/sushazad/Documents/GitHub/ISSE_cleaned/"
filename = "NoiseAutoGen22050.wav"
sample, nsound = scipy.io.wavfile.read(path+filename)
print(sample)
print(nsound.shape)
filename = "ZuzanaNoiseClean.wav"
sample2, nsound2 = scipy.io.wavfile.read(path+filename)
print(sample2)
print(nsound2.shape)
nsound2 = np.reshape(nsound2, (nsound2.shape[0],1))
nsound = np.reshape(nsound, (nsound.shape[0],1))
print(nsound.shape)
print(nsound2.shape)
totNoise = np.vstack((nsound, nsound2))[:,0]
print("Total noise :",totNoise.shape)
totNoise = totNoise[:totBirds.shape[0]]
print("Total noise cropped :",totNoise.shape)
nf, nt, nZxx = scipy.signal.stft(totNoise, fs=sample,window='hann', noverlap=768, nperseg=1024)
nmag = np.abs(nZxx)
nlog = np.log(nmag)
print(nlog.shape)
nlog=nlog[:,:20000]
noiseAloneLog = nlog

22050
(4579785,)
22050
(1389172,)
(4579785, 1)
(1389172, 1)
Total noise : (5968957,)
Total noise cropped : (5968957,)
(513, 23318)


In [42]:
path = "/Users/sushazad/Documents/GitHub/ISSE_cleaned/"
filename = "GGA_Alogging.wav"
sample, nsound = scipy.io.wavfile.read(path+filename)
print(sample)
print(nsound.shape)
lf, lt, lZxx = scipy.signal.stft(nsound, fs=sample,window='hann', noverlap=768, nperseg=1024)
lmag = np.abs(lZxx)
llog = np.log(lmag)
print(llog.shape)
llog = llog[:,:42300]
loggingAloneLog = llog

44100
(10844631,)
(513, 42363)


In [57]:
path = "/Users/sushazad/Documents/GitHub/ISSE_cleaned/"
filename = "GGA_ABirdsLogging.wav"
sample, blsound = scipy.io.wavfile.read(path+filename)
print(sample)
print(blsound.shape)
filename = "../ISSE_Cleaned/GGA_A20160817_060000.wav"
sample2, blsound2 = scipy.io.wavfile.read(path+filename)
print(sample2)
print(blsound2.shape)
blsound = np.reshape(blsound, (blsound.shape[0],1))
blsound2 = np.reshape(blsound2, (blsound2.shape[0],1))
totblsound = np.vstack((blsound, blsound2))[:,0]
blf, blt, blZxx = scipy.signal.stft(totblsound, fs=sample2,window='hann', noverlap=768, nperseg=1024)
blmag = np.abs(blZxx)
bllog = np.log(blmag)
print(bllog.shape)
bllog = bllog[:,:98900]
birdLoggingLog = bllog

44100
(10958850,)
44100
(14376600,)
(513, 98968)


In [58]:
chunked = chunkify(blog)
np.random.shuffle(chunked.T)
print("Shape of chunked birds: ", np.shape(chunked))
nchunked = chunkify(nlog)
np.random.shuffle(chunked.T)
print("Shape of chunked noise: ", np.shape(nchunked))
lchunked = chunkify(llog)
np.random.shuffle(lchunked.T)
print("Shape of chunked logging: ", np.shape(lchunked))
blchunked = chunkify(bllog)
np.random.shuffle(blchunked.T)
print("Shape of chunked birds: ", np.shape(blchunked))

Shape of chunked birds:  (51300, 509)
Shape of chunked noise:  (51300, 200)
Shape of chunked logging:  (51300, 423)
Shape of chunked birds:  (51300, 989)


In [45]:
bird_thresh=int(np.shape(chunked)[1]*0.8)
noise_thresh=int(np.shape(nchunked)[1]*0.8)
logging_thresh=int(np.shape(lchunked)[1]*0.8)
bird_logging_thresh=int(np.shape(blchunked)[1]*0.8)

In [46]:
btrain = chunked[:,:bird_thresh]
print(btrain.shape)
btest = chunked[:,bird_thresh:]
print(btest.shape)
ntrain = nchunked[:,:noise_thresh]
ntest = nchunked[:,noise_thresh:]
ltrain = lchunked[:,:logging_thresh]
print(ltrain.shape)
ltest = lchunked[:,logging_thresh:]
print(ltest.shape)
bltrain = blchunked[:,:bird_logging_thresh]
bltest = blchunked[:,bird_logging_thresh:]

(51300, 407)
(51300, 102)
(51300, 338)
(51300, 85)


In [47]:
all_train = np.hstack((btrain, ntrain, ltrain, bltrain))
print("All train shape:", all_train.shape)

All train shape: (51300, 1247)


In [48]:
from sklearn.decomposition import PCA
pca = PCA(n_components=15)
means = np.mean(all_train, axis=1)
means = np.reshape(means, (means.shape[0], 1))
pca.fit(all_train.T)  
z_pca = pca.fit_transform(all_train.T).T
z_pca.shape

(15, 1247)

In [49]:
bird_z = z_pca[:, :bird_thresh]
noise_z = z_pca[:, bird_thresh:bird_thresh+noise_thresh]
logging_z = z_pca[:, bird_thresh+noise_thresh:bird_thresh+noise_thresh+logging_thresh]
bird_logging_z = z_pca[:, bird_thresh+noise_thresh+logging_thresh:]
print("Shape of noise_z", noise_z.shape)
print("Shape of bird_z", bird_z.shape)
print("Shape of logging_z", logging_z.shape)
print("Shape of bird_logging_z", bird_logging_z.shape)
noise_mean, noise_cov = find_mean_and_cov(noise_z)
bird_mean, bird_cov = find_mean_and_cov(bird_z)
logging_mean, logging_cov = find_mean_and_cov(logging_z)
bird_logging_mean, bird_logging_cov = find_mean_and_cov(bird_logging_z)
    

Shape of noise_z (15, 160)
Shape of bird_z (15, 407)
Shape of logging_z (15, 338)
Shape of bird_logging_z (15, 342)


In [50]:
btest_thresh = btest.shape[1]
ntest_thresh = ntest.shape[1]
ltest_thresh = ltest.shape[1]
bltest_thresh = bltest.shape[1]

all_test = np.hstack((btest,ntest,ltest,bltest))
print("All test shape: ",all_test.shape)

All test shape:  (51300, 313)


In [51]:
true_labels = []
for i in range(btest_thresh):
    true_labels.append(0)
for i in range(ntest_thresh):
    true_labels.append(1)
for i in range(ltest_thresh):
    true_labels.append(2)
for i in range(bltest_thresh):
    true_labels.append(3)
confusion_matrix = np.zeros((4,4))

In [52]:
z_test = pca.components_.dot(all_test-means)

In [53]:
def testing(z_test):
    def bestMatch(sample):
        #Returns 0 (bird) or 1(noise) or 2(logging alone) or 3(birds and logging both)
        sample = np.reshape(sample, (sample.shape[0],1))
        birdz = gaussian(sample,bird_mean, bird_cov)
        noyes = gaussian(sample,noise_mean, noise_cov)
        loggingz = gaussian(sample,logging_mean, logging_cov)
        bird_loggingz = gaussian(sample,bird_logging_mean, bird_logging_cov)
        gaussians = []
        gaussians.append(birdz)
        gaussians.append(noyes)
        gaussians.append(loggingz)
        gaussians.append(bird_loggingz)
        best_match = gaussians.index(max(gaussians))
        return best_match
    n=z_test.shape[1]
    True_noise = 0
    False_bird = 0
    True_bird = 0
    False_noise = 0
    for i in range(n):
        best_match = bestMatch(z_test[:,i])
        true_match = true_labels[i]
        confusion_matrix[true_match][best_match]+=1
    return confusion_matrix

In [54]:
print(len(true_labels))
print(z_test.shape)

313
(15, 313)


In [55]:
conf_mat = testing(z_test)
import pandas as pd
print(pd.DataFrame(conf_mat))

      0     1     2     3
0  92.0  10.0   0.0   0.0
1  21.0  19.0   0.0   0.0
2   0.0   0.0  85.0   0.0
3   0.0   0.0   0.0  86.0
